In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../modules')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
pd.options.display.max_columns=200
pd.options.display.max_rows=100
from encoding import FreqeuncyEncoding
from hyperopt_model_selection_regressor import HyperOptModelSelection,lgbm_space,xgb_space
from custom_estimator_regressor import Estimator
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.externals import joblib
import datetime

### add more historical vars at a member_id level
### think more about neighbouring vars

In [2]:
train=pd.read_csv('train_1.csv')
test=pd.read_csv('test_1.csv')
target=train.amount_spent_per_room_night_scaled

#### loading pre saved data till this point

In [3]:
df=pd.read_csv('temp_df.csv')

In [4]:
df.shape

(488189, 109)

In [5]:
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,checkin_out_delta,booking_checkin_delta,booking_checkout_delta,booking_mon,booking_day,booking_year,booking_dow,booking_week_no,booking_day_of_year,booking_quarter,booking_week_of_yr,checkin_mon,checkin_day,checkin_year,checkin_dow,checkin_week_no,checkin_day_of_year,checkin_quarter,checkin_week_of_yr,child_adult_ratio,num_single_rm_nts_adults,num_single_rm_nts_children,adult_Frac,child_Frac,pax_rm_nts,is_children,cont1,delta_rmnts_checin_in_out_delta,calc_pax_rm_nts,visit_number,last_trip_diff,next_trip_diff,last_total_pax_diff,last_roomnights_diff,last_adult_diff,last_trip_diff_check_in,last_trip_diff_check_out,next_trip_diff_check_in,next_trip_diff_check_out,last_cacl_rmnts_diff_check_in,pax_cum_mean,numberofadults_cum_mean,roomnights_cum_mean,checkin_out_delta_cum_mean,booking_checkin_delta_cum_mean,booking_checkout_delta_cum_mean,visit_number_resort,last_total_pax_diff_resort,last_roomnights_diff_resort,last_adult_diff_resort,pax_cum_mean_resort,numberofadults_cum_mean_resort,roomnights_cum_mean_resort,checkin_out_delta_cum_mean_resort,booking_checkin_delta_cum_mean_resort,booking_checkout_delta_cum_mean_resort,visit_number_state_ind,last_total_pax_diff_state_ind,last_roomnights_diff_state_ind,last_adult_diff_state_ind,pax_cum_mean_state_ind,numberofadults_cum_mean_state_ind,roomnights_cum_mean_state_ind,checkin_out_delta_cum_mean_state_ind,booking_checkin_delta_cum_mean_state_ind,booking_checkout_delta_cum_mean_state_ind,visit_number_state_res,last_total_pax_diff_state_res,last_roomnights_diff_state_res,last_adult_diff_state_res,pax_cum_mean_state_res,numberofadults_cum_mean_state_res,roomnights_cum_mean_state_res,checkin_out_delta_cum_mean_state_res,booking_checkin_delta_cum_mean_state_res,booking_checkout_delta_cum_mean_state_res,resort_id_booking_date_unique,resort_id_checkin_date_unique,resort_id_product_code_unique,resort_id_persontravellingid_unique,resort_id_member_id_unique,resorts_in_a_state,resorts_in_a_region,resorts_in_a_cluster,resorts_in_a_resort_type,person_state_nunique
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,2018-04-05,2018-04-05,2018-04-06,3,1,2,0,46,3,3,3,1,2.0,7.0,3,3,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,C,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,1,0,1,4,5,2018,3,14,95,2,14,4,5,2018,3,14,95,2,14,0.000000,2,0,0.666667,0.0,3,0,0.000000,0,3,14.0,67.0,-1.0,0.0,-2.0,0.0,-1.0,-2.0,-1.0,-1.0,-1.0,2.307692,2.0,3.538462,3.153846,43.230769,46.384615,4618.0,-1,-2,-1,3.427426,3.233536,3.535095,2.585572,49.105936,51.691508,29983.5,-1,-2,0,3.144739,3.469878,3.880679,2.428281,49.639436,52.067716,38688.5,1,-1,0,3.141313,3.131851,3.854757,2.512229,51.311169,53.823397,1458,1401,5,5,5413,6,10,11,8,11471
1,03930f033646d073462b35d411616323597715ac4fc398...,2015-01-23,2015-04-11,2015-04-16,1,1,2,0,46,3,3,4,5,2.0,7.0,5,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,F,A,39fa9ec190eee7b6f4dff1100d6343e10918d044c75eac...,5,78,83,1,23,2015,4,4,23,1,4,4,11,2015,5,15,101,2,15,0.000000,10,0,1.000000,0.0,10,0,0.939759,0,10,1.0,-1.0,-5.0,-1.0,-1.0,-1.0,-1.0,-1.0,69.0,70.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,238.5,-1,-16,-12,2.794118,3.331933,4.831933,2.647059,61.718487,64.365546,1158.5,0,3,0,2.871528,3.351562,4.097222,2.684896,65.642361,68.327257,1665.0,0,2,0,3.155141,3.470836,4.150932,2.697535,58.927240,61.624775,1522,1481,5,5,6488,4,10,11,8,11471
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,2015-01-28,2015-02-01,2015-02-05,1,1,2,0,47,1,5,4,4,2.0,7.0,1,2,F,1,3d1539e56495b6991f0a3ef5a61ca3d03ce4fff7380e9a...,E,A,535fa30d7e25dd8a49f1536779734ec8286108d115da50...,4,4,8,1,28,2015,2,5,28,1,5,2

In [6]:
df['cat_1']= df['resort_id'].astype('str')+'_'+df['memberid'].astype('str')
df['cat_2']= df['resort_id'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_3']= df['resort_id'].astype('str')+'_'+df['state_code_resort'].astype('str')
df['cat_4']= df['resort_id'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_5']= df['resort_id'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_6']= df['state_code_residence'].astype('str')+'_'+df['state_code_residence'].astype('str')
df['cat_7']= df['memberid'].astype('str')+'_'+df['checkin_date'].astype('str')
df['cat_8']= df['memberid'].astype('str')+'_'+df['checkout_date'].astype('str')
df['cat_9']= df['memberid'].astype('str')+'_'+df['cluster_code'].astype('str')
df['cat_10']= df['memberid'].astype('str')+'_'+df['booking_date'].astype('str')


In [7]:
cat_cols=['booking_date', 'checkin_date', 'checkout_date', 'channel_code', 
          'main_product_code', 'persontravellingid',
          'resort_region_code', 'resort_type_code', 'room_type_booked_code', 
          'season_holidayed_code', 'state_code_residence', 'state_code_resort', 
          'member_age_buckets', 'booking_type_code', 'memberid', 'cluster_code', 'reservationstatusid_code',
          'resort_id', 'booking_mon','booking_day', 'booking_year', 'booking_dow', 
          'booking_week_no', 'booking_day_of_year', 'booking_quarter', 
          'booking_week_of_yr', 'checkin_mon', 'checkin_day', 'checkin_year', 'checkin_dow',
          'checkin_week_no', 'checkin_day_of_year', 'checkin_quarter', 'checkin_week_of_yr',
         'cat_1','cat_2','cat_3','cat_4','cat_5','cat_6',
          'cat_7','cat_8','cat_9','cat_10',
         ]

In [8]:
num_cols=['numberofadults', 'numberofchildren','roomnights','total_pax','checkin_out_delta','booking_checkin_delta',
         'booking_checkout_delta','child_adult_ratio','num_single_rm_nts_adults','num_single_rm_nts_children',
         'adult_Frac','child_Frac','pax_rm_nts','is_children','cont1','delta_rmnts_checin_in_out_delta',
         'calc_pax_rm_nts',
         'pax_cum_mean_state_ind', 'checkin_out_delta_cum_mean_resort','visit_number_resort',
          'visit_number_state_ind','visit_number_state_res','pax_cum_mean_resort']

In [9]:
print len(cat_cols), len(num_cols)

44 23


In [10]:
fe=FreqeuncyEncoding(categorical_columns=cat_cols,normalize=False,return_df=True)
df=fe.fit_transform(df)

In [11]:
df.head()

,reservation_id,booking_date,checkin_date,checkout_date,channel_code,main_product_code,numberofadults,numberofchildren,persontravellingid,resort_region_code,resort_type_code,room_type_booked_code,roomnights,season_holidayed_code,state_code_residence,state_code_resort,total_pax,member_age_buckets,booking_type_code,memberid,cluster_code,reservationstatusid_code,resort_id,checkin_out_delta,booking_checkin_delta,booking_checkout_delta,booking_mon,booking_day,booking_year,booking_dow,booking_week_no,booking_day_of_year,booking_quarter,booking_week_of_yr,checkin_mon,checkin_day,checkin_year,checkin_dow,checkin_week_no,checkin_day_of_year,checkin_quarter,checkin_week_of_yr,child_adult_ratio,num_single_rm_nts_adults,num_single_rm_nts_children,adult_Frac,child_Frac,pax_rm_nts,is_children,cont1,delta_rmnts_checin_in_out_delta,calc_pax_rm_nts,visit_number,last_trip_diff,next_trip_diff,last_total_pax_diff,last_roomnights_diff,last_adult_diff,last_trip_diff_check_in,last_trip_diff_check_out,next_trip_diff_check_in,next_trip_diff_check_out,last_cacl_rmnts_diff_check_in,pax_cum_mean,numberofadults_cum_mean,roomnights_cum_mean,checkin_out_delta_cum_mean,booking_checkin_delta_cum_mean,booking_checkout_delta_cum_mean,visit_number_resort,last_total_pax_diff_resort,last_roomnights_diff_resort,last_adult_diff_resort,pax_cum_mean_resort,numberofadults_cum_mean_resort,roomnights_cum_mean_resort,checkin_out_delta_cum_mean_resort,booking_checkin_delta_cum_mean_resort,booking_checkout_delta_cum_mean_resort,visit_number_state_ind,last_total_pax_diff_state_ind,last_roomnights_diff_state_ind,last_adult_diff_state_ind,pax_cum_mean_state_ind,numberofadults_cum_mean_state_ind,roomnights_cum_mean_state_ind,checkin_out_delta_cum_mean_state_ind,booking_checkin_delta_cum_mean_state_ind,booking_checkout_delta_cum_mean_state_ind,visit_number_state_res,last_total_pax_diff_state_res,last_roomnights_diff_state_res,last_adult_diff_state_res,pax_cum_mean_state_res,numberofadults_cum_mean_state_res,roomnights_cum_mean_state_res,checkin_out_delta_cum_mean_state_res,booking_checkin_delta_cum_mean_state_res,booking_checkout_delta_cum_mean_state_res,resort_id_booking_date_unique,resort_id_checkin_date_unique,resort_id_product_code_unique,resort_id_persontravellingid_unique,resort_id_member_id_unique,resorts_in_a_state,resorts_in_a_region,resorts_in_a_cluster,resorts_in_a_resort_type,person_state_nunique,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10
0,07659f3758d8aee27f5a7e2887adeacb67021cb95ada1b...,388,330,348,182950,82844,2,0,43949,95080,70075,196438,1,180864,36820,48294,3,73197,427677,14,172770,20205,5594,1,0,1,36728,16596,115195,73239,9935,1316,114431,9935,43103,17434,119443,64307,9672,1404,127799,9672,0.000000,2,0,0.666667,0.0,3,0,0.000000,0,3,14.0,67.0,-1.0,0.0,-2.0,0.0,-1.0,-2.0,-1.0,-1.0,-1.0,2.307692,2.0,3.538462,3.153846,43.230769,46.384615,4618.0,-1,-2,-1,3.427426,3.233536,3.535095,2.585572,49.105936,51.691508,29983.5,-1,-2,0,3.144739,3.469878,3.880679,2.428281,49.639436,52.067716,38688.5,1,-1,0,3.141313,3.131851,3.854757,2.512229,51.311169,53.823397,1458,1401,5,5,5413,6,10,11,8,11471,1,363,5594,7,5,36820,1,1,5,1
1,03930f033646d073462b35d411616323597715ac4fc398...,252,394,284,230506,82844,2,0,43949,95080,70075,119360,5,180864,36820,46786,2,73197,427677,14,172770,446743,7697,5,78,83,47181,12885,108082,70939,8289,1190,131987,8289,43103,15777,107143,74156,10055,1368,127799,10055,0.000000,10,0,1.000000,0.0,10,0,0.939759,0,10,1.0,-1.0,-5.0,-1.0,-1.0,-1.0,-1.0,-1.0,69.0,70.0,-1.0,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,238.5,-1,-16,-12,2.794118,3.331933,4.831933,2.647059,61.718487,64.365546,1158.5,0,3,0,2.871528,3.351562,4.097222,2.684896,65.642361,68.327257,1665.0,0,2,0,3.155141,3.470836,4.150932,2.697535,58.927240,61.624775,1522,1481,5,5,6488,4,10,11,8,11471,1,518,7697,8,8,36820,1,1,5,1
2,d145a32920e6587ad95bfe299d80c0affa268220535aaf...,304,272,244,230506,82844,2,0,45062,205751,38152,119360,4,180864,36820,85361,2,73197,427677,14,85361,4

In [12]:
train=df[df.reservation_id.isin(train.reservation_id)]
test=df[df.reservation_id.isin(test.reservation_id)]
train.drop('reservation_id',axis=1,inplace=True)
test.drop('reservation_id',axis=1,inplace=True)
print train.shape, test.shape

(341424, 118) (146765, 118)


/home/ubuntu/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
del df

In [14]:
import gc
gc.collect()

0

In [18]:
print train.shape, test.shape

(341424, 118) (146765, 118)


In [19]:
params={'bagging_freq': 5, 'boosting_type': 'gbdt', 'colsample_bytree': 0.4000000000000001,
 'learning_rate': 0.01, 'min_child_weight': 300.0, 'n_estimators': 15000,
 'n_jobs': -1, 'num_leaves': 64,'subsample': 1.0}

In [20]:
mod=Estimator(model=LGBMRegressor(**params),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=100,n_splits=5,early_stopping_rounds=200,shuffle=True)

In [21]:
%%time
oof=mod.fit_transform(train.values,target.values)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 1.01313	valid_0's l2: 1.02643	valid_1's rmse: 1.01119	valid_1's l2: 1.02251
[200]	valid_0's rmse: 0.987815	valid_0's l2: 0.975778	valid_1's rmse: 0.985128	valid_1's l2: 0.970477
[300]	valid_0's rmse: 0.97713	valid_0's l2: 0.954783	valid_1's rmse: 0.973119	valid_1's l2: 0.94696
[400]	valid_0's rmse: 0.971944	valid_0's l2: 0.944675	valid_1's rmse: 0.966247	valid_1's l2: 0.933633
[500]	valid_0's rmse: 0.969043	valid_0's l2: 0.939043	valid_1's rmse: 0.961496	valid_1's l2: 0.924475
[600]	valid_0's rmse: 0.967223	valid_0's l2: 0.935521	valid_1's rmse: 0.957761	valid_1's l2: 0.917306
[700]	valid_0's rmse: 0.966089	valid_0's l2: 0.933328	valid_1's rmse: 0.954646	valid_1's l2: 0.911348
[800]	valid_0's rmse: 0.965287	valid_0's l2: 0.93178	valid_1's rmse: 0.951861	valid_1's l2: 0.90604
[900]	valid_0's rmse: 0.96471	valid_0's l2: 0.930665	valid_1's rmse: 0.949311	valid_1's l2: 0.901192
[1000]	valid_0's rmse: 0.964

[1800]	valid_0's rmse: 0.954642	valid_0's l2: 0.911341	valid_1's rmse: 0.933807	valid_1's l2: 0.871995
[1900]	valid_0's rmse: 0.954596	valid_0's l2: 0.911253	valid_1's rmse: 0.932061	valid_1's l2: 0.868738
[2000]	valid_0's rmse: 0.95455	valid_0's l2: 0.911166	valid_1's rmse: 0.93033	valid_1's l2: 0.865514
[2100]	valid_0's rmse: 0.954516	valid_0's l2: 0.9111	valid_1's rmse: 0.928649	valid_1's l2: 0.86239
[2200]	valid_0's rmse: 0.954479	valid_0's l2: 0.911031	valid_1's rmse: 0.926959	valid_1's l2: 0.859253
[2300]	valid_0's rmse: 0.954437	valid_0's l2: 0.910951	valid_1's rmse: 0.925318	valid_1's l2: 0.856213
[2400]	valid_0's rmse: 0.954388	valid_0's l2: 0.910856	valid_1's rmse: 0.92365	valid_1's l2: 0.853129
[2500]	valid_0's rmse: 0.954388	valid_0's l2: 0.910857	valid_1's rmse: 0.922046	valid_1's l2: 0.850169
[2600]	valid_0's rmse: 0.954364	valid_0's l2: 0.91081	valid_1's rmse: 0.92045	valid_1's l2: 0.847228
[2700]	valid_0's rmse: 0.954381	valid_0's l2: 0.910842	valid_1's rmse: 0.918857	v

In [19]:
print mod.cv_scores, mod.avg_cv_score,np.std(mod.cv_scores)

[0.9626701677143645, 0.9640278675884069, 0.9543578323627437, 0.9718833217355385, 0.9687809081457903] 0.9643440195093689 0.005985862083705572


In [20]:
mod2=Estimator(model=LGBMRegressor(**params),n_jobs=-1,
                          validation_scheme='KFold',
              random_state=200,n_splits=5,early_stopping_rounds=200,shuffle=True)

In [21]:
%%time
oof2=mod2.fit_transform(train.values,target.values)

Training until validation scores don't improve for 200 rounds.
[100]	valid_0's rmse: 1.01107	valid_0's l2: 1.02226	valid_1's rmse: 1.01126	valid_1's l2: 1.02264
[200]	valid_0's rmse: 0.986869	valid_0's l2: 0.97391	valid_1's rmse: 0.984984	valid_1's l2: 0.970194
[300]	valid_0's rmse: 0.976836	valid_0's l2: 0.954208	valid_1's rmse: 0.972918	valid_1's l2: 0.946569
[400]	valid_0's rmse: 0.972066	valid_0's l2: 0.944913	valid_1's rmse: 0.96602	valid_1's l2: 0.933196
[500]	valid_0's rmse: 0.969394	valid_0's l2: 0.939724	valid_1's rmse: 0.961253	valid_1's l2: 0.924008
[600]	valid_0's rmse: 0.967693	valid_0's l2: 0.936429	valid_1's rmse: 0.957515	valid_1's l2: 0.916834
[700]	valid_0's rmse: 0.966655	valid_0's l2: 0.934421	valid_1's rmse: 0.954404	valid_1's l2: 0.910886
[800]	valid_0's rmse: 0.965924	valid_0's l2: 0.933009	valid_1's rmse: 0.951629	valid_1's l2: 0.905598
[900]	valid_0's rmse: 0.965398	valid_0's l2: 0.931993	valid_1's rmse: 0.949125	valid_1's l2: 0.900838
[1000]	valid_0's rmse: 0.

[1700]	valid_0's rmse: 0.96399	valid_0's l2: 0.929277	valid_1's rmse: 0.933272	valid_1's l2: 0.870998
[1800]	valid_0's rmse: 0.963967	valid_0's l2: 0.929232	valid_1's rmse: 0.931501	valid_1's l2: 0.867694
[1900]	valid_0's rmse: 0.963929	valid_0's l2: 0.929159	valid_1's rmse: 0.929787	valid_1's l2: 0.864504
[2000]	valid_0's rmse: 0.963898	valid_0's l2: 0.929099	valid_1's rmse: 0.928076	valid_1's l2: 0.861324
[2100]	valid_0's rmse: 0.963856	valid_0's l2: 0.929019	valid_1's rmse: 0.926379	valid_1's l2: 0.858177
[2200]	valid_0's rmse: 0.963842	valid_0's l2: 0.928992	valid_1's rmse: 0.924705	valid_1's l2: 0.855079
[2300]	valid_0's rmse: 0.963823	valid_0's l2: 0.928955	valid_1's rmse: 0.923068	valid_1's l2: 0.852054
[2400]	valid_0's rmse: 0.963798	valid_0's l2: 0.928907	valid_1's rmse: 0.921466	valid_1's l2: 0.849099
[2500]	valid_0's rmse: 0.963773	valid_0's l2: 0.928858	valid_1's rmse: 0.919837	valid_1's l2: 0.846101
[2600]	valid_0's rmse: 0.963743	valid_0's l2: 0.9288	valid_1's rmse: 0.918

In [22]:
print mod2.cv_scores, mod2.avg_cv_score,np.std(mod2.cv_scores)

[0.963788839162787, 0.9608491800067562, 0.9637146591502748, 0.9644156925283296, 0.9692155572066771] 0.9643967856109649 0.002707076581052156


In [23]:
preds1=mod.predict(test.values)
preds2=mod2.predict(test.values)

custom_estimator_regressor.py:108: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.mean(np.column_stack((est.predict(x) for est in self.fitted_models)), axis=1)


In [24]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds1})
sol.to_csv('preds_lgb_118_seed_100.csv',index=False)

In [25]:
sub=pd.read_csv('sample_submission.csv')
res_ids=pd.read_csv('test.csv',usecols=['reservation_id']).reservation_id
sol=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':preds2})
sol.to_csv('preds_lgb_118_seed_200.csv',index=False)

In [26]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof2})
oof_df.to_csv('OOF_lgb_118_seed_200.csv',index=False)

In [27]:
res_ids=pd.read_csv('train.csv',usecols=['reservation_id']).reservation_id
oof_df=pd.DataFrame({'reservation_id':res_ids,'amount_spent_per_room_night_scaled':oof})
oof_df.to_csv('OOF_lgb_118_seed_100.csv',index=False)